## This file helps to quickly write code and test it and alter on we will put in respective files as per modular coding

In [49]:

import langchain
print(langchain.__version__)
import sys
print(sys.executable)


1.2.0
/Users/amitpal/Desktop/GitRepos/RAG_ML_chatbot/venv/bin/python


In [50]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import os
load_dotenv()

True

In [51]:
def file_loader(path):
    loader=DirectoryLoader(
        path,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )    
    documents=loader.load()
    return documents

In [52]:
extracted_docs=file_loader("data/")

In [53]:
def chunking_data(split_documents):
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
    )
    chunk_data=text_splitter.split_documents(split_documents)
    return chunk_data

In [54]:
chunk_data=chunking_data(extracted_docs)
print(len(chunk_data))

1680


In [55]:
def get_embedding():
    embedding=OpenAIEmbeddings(model="text-embedding-3-small")
    return embedding

In [56]:
embedding=get_embedding()

In [57]:
vector_store=FAISS.from_documents(documents=chunk_data,embedding=embedding)
vector_store

In [58]:
retriever=vector_store.as_retriever(search_type="similarity",search_kwargs={"k":3})
#response=retriever.invoke("what is supervised machine learning")
#response

In [59]:
model=ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5
)

In [60]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following using below context.
context:
{context}

Question:
{question}
""")

rag_chain=(
    {
        "context":retriever,
        "question":RunnablePassthrough()
    }|prompt|model
)

rag_chain.invoke("What is this supervised learning learning")




AIMessage(content='Supervised learning is a type of machine learning where a model is trained on a labeled dataset. In this approach, the algorithm learns to map input data to the corresponding output labels by finding patterns in the training data. The goal is to make accurate predictions or classifications on new, unseen data based on the learned relationships. Supervised learning is commonly used in applications such as classification and regression tasks, where the output is known during the training phase.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 684, 'total_tokens': 772, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CuZakJIuTzmQ